# Konvertering av Bergens registeret



In [1]:
from hentOgVaskData import hent_og_vask_data
from bergens_kolonner_med_verdi import bergens_kolonner_med_verdi
import polars as pl
pl.Config.set_tbl_rows(30)

polars.config.Config

In [2]:
'''

1. Leser inn Data exportert i fra Bergens Access database.
   I fra den tabellen som heter `tblInnmeldte`. Dette gjøres ved å benytte funksjonen `hent_og_vask_data()`. Denne funksjonen er også hardkodet med de som har dobbelt medlemskap mellom registerene. Disse blir satt til "venn" i stedet for medlem av Bergens registeret.
'''

# leser inn Data exportert i fra Bergens Access database
# I fra den tabellen som heter "tblInnmeldte"
df = hent_og_vask_data()
# Medlems sjekk, det er viktig at dette tallet er nokså likt som rapportert in til statsforvalteren 282
print("Antall medlemmer i Bergens registeret -->", df.filter(pl.col("Kategori").str.starts_with("M"))["MedlemID"])
#print("Alle data i fra excel fil -->", df)

Could not determine dtype for column 43, falling back to string


Antall medlemmer i Bergens registeret --> shape: (279,)
Series: 'MedlemID' [i64]
[
	14570
	15536
	14800
	14720
	13073
	14015
	15442
	14791
	11208
	13277
	15537
	12968
	14462
	11209
	11210
	…
	15460
	11692
	13649
	14744
	12506
	14482
	14481
	14483
	14063
	13622
	14432
	11027
	14294
	13451
	15638
]


In [3]:
# Statestikk over dataene i fra Bergen

summary_wide = df.describe()
# 2. Reshape it into a clean, readable format using the corrected code
summary_clean = (
    summary_wide
    # Use the modern .unpivot() instead of .melt()
    # Use 'statistic' as the index column
    .unpivot(index="statistic", variable_name="column_name", value_name="value")
    
    # The pivot operation also needs to reference 'statistic'
    .pivot(index="column_name", columns="statistic", values="value")
)
pl.Config.set_tbl_cols(-1)
pl.Config.set_tbl_rows(-1)
print("\n--- 2. Clean, Pivoted Statistics (Corrected and Easy to Read!) ---")
print(summary_clean)
pl.Config.set_tbl_cols(10)
pl.Config.set_tbl_rows(10)


--- 2. Clean, Pivoted Statistics (Corrected and Easy to Read!) ---
shape: (46, 10)
┌──────────┬───────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┐
│ column_n ┆ count ┆ null_co ┆ mean    ┆ std     ┆ min     ┆ 25%     ┆ 50%     ┆ 75%     ┆ max     │
│ ame      ┆ ---   ┆ unt     ┆ ---     ┆ ---     ┆ ---     ┆ ---     ┆ ---     ┆ ---     ┆ ---     │
│ ---      ┆ str   ┆ ---     ┆ str     ┆ str     ┆ str     ┆ str     ┆ str     ┆ str     ┆ str     │
│ str      ┆       ┆ str     ┆         ┆         ┆         ┆         ┆         ┆         ┆         │
╞══════════╪═══════╪═════════╪═════════╪═════════╪═════════╪═════════╪═════════╪═════════╪═════════╡
│ MedlemID ┆ 577.0 ┆ 0.0     ┆ 13871.2 ┆ 1376.27 ┆ 11018.0 ┆ 12720.0 ┆ 14443.0 ┆ 14764.0 ┆ 15664.0 │
│          ┆       ┆         ┆ 0970537 ┆ 4581994 ┆         ┆         ┆         ┆         ┆         │
│          ┆       ┆         ┆ 2617    ┆ 6433    ┆         ┆         ┆         ┆         ┆         │
│ Innme

/tmp/ipykernel_85187/1052446714.py:12: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  .pivot(index="column_name", columns="statistic", values="value")


polars.config.Config

In [4]:
'''
df.select("MedlemID", "SamID", "FORNAVN", "EFTERNAVN", "Personnummer"
         ).sort("SamID"
               )#.filter(pl.col("Personnummer").str.len_chars() == 6)
'''

'\ndf.select("MedlemID", "SamID", "FORNAVN", "EFTERNAVN", "Personnummer"\n         ).sort("SamID"\n               )#.filter(pl.col("Personnummer").str.len_chars() == 6)\n'

# Konvertering utrekk

In [6]:
'''
2. Les inn hjelpe tabeller for konvertering mellom Bergen Oslo:
	1. Sivil status(Kollonne "Sivilstand" i Bergen)
	2.  Medlems status(Kolonne "Kategori" i Bergen)
'''

# Leser inn hjelpe tabeller for Medlemstyper og Sivilstatus
df_sivil = pl.read_excel("sivil_status.xlsx")
df_medlems_type = pl.read_excel("kategorier.xlsx")

print(df_medlems_type)
print(df_sivil)

shape: (13, 3)
┌──────────────────┬─────────────┬─────────────────────────────────┐
│ Access Kattegori ┆ Cornerstone ┆ Beskirvelse                     │
│ ---              ┆ ---         ┆ ---                             │
│ str              ┆ str         ┆ str                             │
╞══════════════════╪═════════════╪═════════════════════════════════╡
│ DB               ┆ null        ┆ ? dåpsbarn                      │
│ DF               ┆ null        ┆ ? dåps foreldre                 │
│ DØD              ┆ null        ┆ Blir satt til status død, egen… │
│ FLY              ┆ null        ┆ ? Tror det betyr flyttet, kan … │
│ M                ┆ Medlem      ┆ null                            │
│ …                ┆ …           ┆ …                               │
│ O                ┆ null        ┆ Organisasjon, Egen behandling   │
│ P                ┆ null        ┆ ? p                             │
│ D                ┆ null        ┆ null                            │
│ UTM              

In [7]:
df = bergens_kolonner_med_verdi(df, "Anmerkning")
#df_test.select("FORNAVN", "EFTERNAVN", "Anmerkning")
df

MedlemID,Innmeldingsdato,Er den innmeldte døpt ?,Menighet,Fødested,…,Søk,UtmeldtDato,DødsDato,Kommune,KID-nr
i64,date,bool,str,str,…,str,date,str,str,str
13812,null,false,"""Bergen""",null,…,null,null,null,"""AURLAND""",null
14571,2002-09-30,false,"""Bergen""",null,…,null,2015-08-15,null,null,null
14570,2002-09-30,false,null,null,…,null,null,null,null,null
15535,null,false,null,null,…,null,null,null,null,null
14642,null,false,null,null,…,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…
14641,null,false,null,null,…,null,null,null,null,null
15638,2023-03-04,false,"""Bergen""","""Fitjar""",…,null,null,null,null,null
12529,null,false,"""Bergen""",null,…,null,1998-06-24,null,"""SVEIO""",null


In [8]:
'''
3. Tilpasser hele Bergens uttrekket til verdier i kolonnene som er gyldige for Oslo registeret. Dette er utgangspunkt for import i Cornerstone.
   Tabellen `df_konverterings_tabell` skal brukes til å import til Cornerstone. Alle forekomstene  er nye personer i Cornerstone
'''


# The mapping dictionary only defines 'M' and 'K'
gender_map = {
    "M": "Mann",
    "K": "Kvinne"
}

DATE_FORMAT :str = "%d.%m.%Y"
df_konverterings_tabell = (
    df
        .join(
            # 1. First join: rename "Cornerstone" to be specific
            df_medlems_type.select([
                "Access Kattegori", 
                pl.col("Cornerstone").alias("Kategori_Cornerstone")
            ]),
            left_on="Kategori",
            right_on="Access Kattegori",
            how="left"
        )
        .join(
            # 2. Second join: rename "Cornerstone" here as well
            df_sivil.select([
                "Access Sivilstatus", 
                pl.col("Cornerstone").alias("Sivilstand_Cornerstone")
            ]),
            left_on="Sivilstand",
            right_on="Access Sivilstatus",
            how="left"
        )
        .select(
        pl.col("MedlemID"), # 
        pl.lit(None, dtype=pl.Int64).alias("Cornerstone-ID"), # Dummy collumn, Used with handling of dublets
        pl.col("Innmeldingsdato").dt.strftime(DATE_FORMAT),
        pl.col("Er den innmeldte døpt ?"),
        pl.col("Menighet"),
        pl.col("Fødested"),
        pl.col("Dåpsdato").dt.strftime(DATE_FORMAT),
        pl.col("Ant barn under 15 år"),
        pl.col("Dåpskirke/sted"),
        pl.col("Fødselsdato").dt.strftime(DATE_FORMAT),
        pl.col("Personnummer"),
        pl.col("EFTERNAVN"),
        pl.col("FORNAVN"),
        pl.col("Stilling"),
        pl.col("ADRESSE"),
        pl.col("E-postadresse"),
        pl.col("POSTNR"),
        pl.col("Og"),
        pl.col("SAM_FOR"),
        pl.col("SAM_EFTER"),
        pl.col("Utsendelse"),
        pl.col("Utsendelses kode"),
        pl.col("Telefon priv"),
        pl.col("Telefon mob"),
        pl.col("Sivilstand_Cornerstone"), # Lagt til 
        pl.col("Sivilstand"),
        pl.col("Trossamfunn"),
        pl.col("Gift?"),
        pl.col("Utsendelsesformat"),
        pl.col("Ektemakens trossamfunn"),
        pl.col("Anmerkning"),
        pl.col("Kategori_Cornerstone"), # Lagt til
        pl.col("Kategori"),
        pl.when(pl.col("Giro") == True) # The condition: is the value True?
            .then(pl.lit("Giro"))           # The result if True
            .otherwise(None)                # The result for all other cases (False and Null)
            .alias("Giro"),             # Give the new column a name
        pl.col("RegDato").dt.strftime(DATE_FORMAT),
        pl.col("SamID"),
        pl.col("FarID"),
        pl.col("MorID"),
        pl.col("Kjønn").replace_strict(
            gender_map,
            default=None # Set the default for non-matches to null        
        ).alias("Kjønn"),
        pl.col("Svart?"),
        pl.col("Faddere"),
        pl.col("Er mor og far gift?"),
        pl.col("Prest"),
        pl.col("Dåpsattest"),
        pl.col("Søk"),
        pl.col("UtmeldtDato").dt.strftime(DATE_FORMAT),
        #pl.col("DødsDato").dt.strftime(DATE_FORMAT), Dødsdato er en string
        #pl.col("DødsDato"),
        pl.when(pl.col("DødsDato").is_null() &
               (pl.col("Kategori") == "DØD")
               ) # The condition: is the value True?
            .then(pl.lit("01.01.2025"))           # The result if True
            .otherwise(None)                # The result for all other cases (False and Null)
            .alias("DødsDato"),             # Give the new column a name
        pl.col("Kommune"),
        pl.col("KID-nr")

    )
)#.filter(pl.col("DødsDato").is_not_null())
pl.Config.set_tbl_cols(-1)
#konverterings_tabell.filter(pl.col("Kategori") == "DØD")

df_konverterings_tabell

MedlemID,Cornerstone-ID,Innmeldingsdato,Er den innmeldte døpt ?,Menighet,Fødested,Dåpsdato,Ant barn under 15 år,Dåpskirke/sted,Fødselsdato,Personnummer,EFTERNAVN,FORNAVN,Stilling,ADRESSE,E-postadresse,POSTNR,Og,SAM_FOR,SAM_EFTER,Utsendelse,Utsendelses kode,Telefon priv,Telefon mob,Sivilstand_Cornerstone,Sivilstand,Trossamfunn,Gift?,Utsendelsesformat,Ektemakens trossamfunn,Anmerkning,Kategori_Cornerstone,Kategori,Giro,RegDato,SamID,FarID,MorID,Kjønn,Svart?,Faddere,Er mor og far gift?,Prest,Dåpsattest,Søk,UtmeldtDato,DødsDato,Kommune,KID-nr
i64,i64,str,bool,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,str,str,str,str,str,str,i64,i64,i64,str,bool,str,bool,str,bool,str,str,str,str,str
13812,null,null,false,"""Bergen""",null,null,2,null,"""20.02.1962""",null,"""Aeschlimann""","""Martin Daniel Peter""",null,"""Tørsdalsvegen 573""",null,"""5574""",null,null,null,null,null,"""52 76 48 08""",null,null,null,"""Intet""",false,null,null,""" Felter i fra det gammle Berge…",null,"""DF""",null,"""31.05.2025""",13617,null,null,"""Mann""",false,null,false,null,false,null,null,null,"""AURLAND""",null
14571,null,"""30.09.2002""",false,"""Bergen""",null,null,null,null,"""28.02.1992""","""28029245543""","""Agdal""","""Bendik Olai""",null,"""Fjellien 3""",null,"""5019""",null,null,null,null,null,null,null,"""Singel""","""Guttebarn""","""Kristensamfunnet""",false,null,null,""" Felter i fra det gammle Berge…",null,"""UTM""",null,"""12.03.2019""",null,14568,15533,"""Mann""",false,null,false,null,false,null,"""15.08.2015""",null,null,null
14570,null,"""30.09.2002""",false,null,null,null,null,null,"""27.06.1997""","""27069744422""","""Agdal""","""Oda Johanna""",null,"""Professot Keysers gate 4""",null,"""5007""",null,null,null,null,null,null,null,"""Singel""","""Pikebarn""","""Kristensamfunnet""",false,null,null,""" Felter i fra det gammle Berge…","""Medlem""","""MB""",null,"""18.12.2014""",null,14568,15535,"""Kvinne""",false,null,false,null,false,null,null,null,null,null
15535,null,null,false,null,null,null,null,null,null,null,"""Agdal""","""Rita""",null,"""Professor Keysers gate 4""",null,"""5007""",null,null,null,"""UB""","""Bn""",null,null,null,null,null,false,"""E""",null,""" Felter i fra det gammle Berge…",null,"""V""",null,"""16.02.2025""",null,null,null,null,false,null,false,null,false,null,null,null,null,null
14642,null,null,false,null,null,null,null,null,null,null,"""Aggerholm""","""Sissel""",null,"""Starefossveien 9""",null,"""5019""",null,null,null,null,null,null,null,null,null,"""Den norske Kirke""",false,null,null,""" Felter i fra det gammle Berge…",null,"""V""",null,"""31.05.2014""",null,null,null,null,false,null,false,null,false,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
14641,null,null,false,null,null,null,null,null,null,null,"""Aanonsen""","""Per Helge""",null,"""Sogn Jord- og Hagebrukskule""",null,"""5745""",null,null,null,null,null,null,null,null,null,null,false,null,null,""" Felter i fra det gammle Berge…",null,"""V""",null,"""31.05.2014""",null,null,null,null,false,null,false,null,false,null,null,null,null,null
15638,null,"""04.03.2023""",false,"""Bergen""","""Fitjar""",null,null,null,"""29.06.1971""","""29067148531""","""Aasheim""","""Frode""",null,"""Bømlavegen 92""",null,"""5437""",null,null,null,null,null,null,null,"""Gift""","""Gift mann""","""Kristensamfunnet""",true,"""P""","""Kristensamfunnet""",""" Felter i fra det gammle Berge…","""Medlem""","""M""",null,"""16.06.2023""",15637,null,null,"""Mann""",false,null,false,null,false,null,null,null,null,null
12529,null,null,false,"""Bergen""",null,null,null,null,"""04.04.1981""",null,"""Aasland""","""Carl Magnus""",null,null,null,"""5555""",null,null,null,null,null,null,null,null,null,null,false,null,null,""" Felter i fra det gammle Berge…",null,"""UTM""","""Giro""","""26.01.1999""",null,null,null,"""Mann""",false,null,false,null,false,null,"""24.

In [9]:
#konverterings_tabell.write_excel("konverterings_tabell.xlsx")

# Konvertering av dubletter

In [10]:
# 4. 

'''
4. Henter ut data som er uten dubletter mellom registerene.
   disse konverteres som nye forekomster som ikke finnes i Oslo registeret.
   for å gjøre denne jobben må vi ta bort de personer som er dubletter med Oslo registeret.
   Tabellen `df_konverterings_tabell_uten_dubletter` skal brukes til å import til Cornerstone. Alle forekomstene er nye personer i Cornerstone
'''



df_combined_duplicates = pl.read_excel("DubletterOsloBergen.xlsx")

df_konverterings_tabell_uten_dubletter = (
    df_konverterings_tabell
        .join(
            df_combined_duplicates
                .filter(pl.col("Menighet") == "OSLO") # Det spiller ingen rollo om en velger OSLO eller BERGEN her
                .select(pl.col(["MedlemID"])),
        on="MedlemID",
        how="anti"
    )
)

df_konverterings_tabell_uten_dubletter

MedlemID,Cornerstone-ID,Innmeldingsdato,Er den innmeldte døpt ?,Menighet,Fødested,Dåpsdato,Ant barn under 15 år,Dåpskirke/sted,Fødselsdato,Personnummer,EFTERNAVN,FORNAVN,Stilling,ADRESSE,E-postadresse,POSTNR,Og,SAM_FOR,SAM_EFTER,Utsendelse,Utsendelses kode,Telefon priv,Telefon mob,Sivilstand_Cornerstone,Sivilstand,Trossamfunn,Gift?,Utsendelsesformat,Ektemakens trossamfunn,Anmerkning,Kategori_Cornerstone,Kategori,Giro,RegDato,SamID,FarID,MorID,Kjønn,Svart?,Faddere,Er mor og far gift?,Prest,Dåpsattest,Søk,UtmeldtDato,DødsDato,Kommune,KID-nr
i64,i64,str,bool,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,str,str,str,str,str,str,i64,i64,i64,str,bool,str,bool,str,bool,str,str,str,str,str
13812,null,null,false,"""Bergen""",null,null,2,null,"""20.02.1962""",null,"""Aeschlimann""","""Martin Daniel Peter""",null,"""Tørsdalsvegen 573""",null,"""5574""",null,null,null,null,null,"""52 76 48 08""",null,null,null,"""Intet""",false,null,null,""" Felter i fra det gammle Berge…",null,"""DF""",null,"""31.05.2025""",13617,null,null,"""Mann""",false,null,false,null,false,null,null,null,"""AURLAND""",null
14571,null,"""30.09.2002""",false,"""Bergen""",null,null,null,null,"""28.02.1992""","""28029245543""","""Agdal""","""Bendik Olai""",null,"""Fjellien 3""",null,"""5019""",null,null,null,null,null,null,null,"""Singel""","""Guttebarn""","""Kristensamfunnet""",false,null,null,""" Felter i fra det gammle Berge…",null,"""UTM""",null,"""12.03.2019""",null,14568,15533,"""Mann""",false,null,false,null,false,null,"""15.08.2015""",null,null,null
14570,null,"""30.09.2002""",false,null,null,null,null,null,"""27.06.1997""","""27069744422""","""Agdal""","""Oda Johanna""",null,"""Professot Keysers gate 4""",null,"""5007""",null,null,null,null,null,null,null,"""Singel""","""Pikebarn""","""Kristensamfunnet""",false,null,null,""" Felter i fra det gammle Berge…","""Medlem""","""MB""",null,"""18.12.2014""",null,14568,15535,"""Kvinne""",false,null,false,null,false,null,null,null,null,null
15535,null,null,false,null,null,null,null,null,null,null,"""Agdal""","""Rita""",null,"""Professor Keysers gate 4""",null,"""5007""",null,null,null,"""UB""","""Bn""",null,null,null,null,null,false,"""E""",null,""" Felter i fra det gammle Berge…",null,"""V""",null,"""16.02.2025""",null,null,null,null,false,null,false,null,false,null,null,null,null,null
14642,null,null,false,null,null,null,null,null,null,null,"""Aggerholm""","""Sissel""",null,"""Starefossveien 9""",null,"""5019""",null,null,null,null,null,null,null,null,null,"""Den norske Kirke""",false,null,null,""" Felter i fra det gammle Berge…",null,"""V""",null,"""31.05.2014""",null,null,null,null,false,null,false,null,false,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
14641,null,null,false,null,null,null,null,null,null,null,"""Aanonsen""","""Per Helge""",null,"""Sogn Jord- og Hagebrukskule""",null,"""5745""",null,null,null,null,null,null,null,null,null,null,false,null,null,""" Felter i fra det gammle Berge…",null,"""V""",null,"""31.05.2014""",null,null,null,null,false,null,false,null,false,null,null,null,null,null
15638,null,"""04.03.2023""",false,"""Bergen""","""Fitjar""",null,null,null,"""29.06.1971""","""29067148531""","""Aasheim""","""Frode""",null,"""Bømlavegen 92""",null,"""5437""",null,null,null,null,null,null,null,"""Gift""","""Gift mann""","""Kristensamfunnet""",true,"""P""","""Kristensamfunnet""",""" Felter i fra det gammle Berge…","""Medlem""","""M""",null,"""16.06.2023""",15637,null,null,"""Mann""",false,null,false,null,false,null,null,null,null,null
12529,null,null,false,"""Bergen""",null,null,null,null,"""04.04.1981""",null,"""Aasland""","""Carl Magnus""",null,null,null,"""5555""",null,null,null,null,null,null,null,null,null,null,false,null,null,""" Felter i fra det gammle Berge…",null,"""UTM""","""Giro""","""26.01.1999""",null,null,null,"""Mann""",false,null,false,null,false,null,"""24.

In [11]:

# Dublett sjekk for å se om det virkelig stemmer

from find_fuzzy_duplicates import find_fuzzy_duplicates

df_test_duplicates = find_fuzzy_duplicates(
    df=df_konverterings_tabell_uten_dubletter,
    columns=["FORNAVN", "EFTERNAVN"],
    columns_to_view=["Cornerstone-ID", "MedlemID", "Kategori" ,"ADRESSE", "POSTNR", "Personnummer"],
    #columns_to_view=None,
    similarity_threshold=90,
    #blocking_column="Postnummer"
)
#duplicates_found.select("fuzzy_group","MedlemID", "FORNAVN", "EFTERNAVN", "ADRESSE", "POSTNR")
df_test_duplicates


Step 1/3: Creating candidate pairs via cross join... (this can be very slow)
 -> Found 131,841 pairs to compare.


Step 2/3: Calculating Similarities:   0%|          | 0/131841 [00:00<?, ?it/s]

Step 3/3: Found 2 duplicate pairs. Building graph and assigning group IDs...


 -> Processing groups:   0%|          | 0/512 [00:00<?, ?it/s]

fuzzy_group,FORNAVN,EFTERNAVN,Cornerstone-ID,MedlemID,Kategori,ADRESSE,POSTNR,Personnummer
i64,str,str,i64,i64,str,str,str,str
0,"""Kari""","""Berger""",null,14439,"""M""","""Gjerdåsvegen 4""","""5428""","""31085332832"""
0,"""Kari Anna""","""Berger""",null,15592,"""M""","""Nordahl Rolfsensvei 22A""","""5094""","""22077849479"""
1,"""Lisbeth""","""Sunde""",null,15578,"""V""","""Løvåsveien 70""","""5145""",null
1,"""Lisbeth""","""Sundt""",null,13172,"""V""","""Gimlebakken 18""","""5052""",null


# Konvertering av dubletter.
- Hente komentarer fra Cornerstone og sette de øverst i komentarfeltet. Komentarer fra bergen settes under.

In [12]:
'''
5. Henter kommentarer fra Cornerstone. Kommentarene hentes i fra et regneark som er eksportert i fra Cornerstone. Alle i cornerstone som har en kommentar på seg. Henter bare kommentarer for de som er dubletter
'''

# Legger inn utrekk av komentarer fra Cornerstone til "manuell" liste av dubletter
# Disse vil bli brukt 
print("df_combined_duplicates", df_combined_duplicates.shape)
df_comments = pl.read_excel("kommentarerFraCornerstone.xlsx")
print("df_comments", df_comments.shape)

df_dublett_komentarer = (
    df_combined_duplicates
    .join(
        df_comments.select(
            pl.col(["Cornerstone-ID", "Kommentar"])
        ),
        on="Cornerstone-ID",
        how="left"
    )
)

df_dublett_komentarer

df_combined_duplicates (126, 13)
df_comments (818, 5)


Menighet,fuzzy_group,Fornavn,Etternavn,Mellomnavn,Cornerstone-ID,MedlemID,Medlemstype 1,Adresse,Postnummer,Innmeldt dato 1,Utmeldt dato 1,Fødselsnummer,Kommentar
str,i64,str,str,str,i64,i64,str,str,str,str,str,str,str
"""OSLO""",0,"""Janicke""","""Akerø""",null,2057856,15536,null,"""Svend Foyns g 69A""","""4016""",null,null,"""07115936284""",null
"""BERGEN""",0,"""Janicke""","""Akerø""",null,2057856,15536,"""M""","""Richard Nordraaks gate 9""","""5018""",null,null,"""07115936284""",null
"""OSLO""",2,"""Gry Veronica""","""Alsos""",null,4681949,14800,null,"""Solåsen 33B""","""5223""",null,null,"""22077028874""","""-- 24.08.2023 Folkeregister op…"
"""BERGEN""",2,"""Gry Veronica""","""Alsos""",null,4681949,14800,"""M""","""Solåsen 33B""","""5223""","""07.10.2006""",null,"""22077028874""","""-- 24.08.2023 Folkeregister op…"
"""OSLO""",6,"""Milla Kristine Hauglin""","""Aukrust""","""Hauglin""",3567891,11780,"""Medlem""","""Einars Vei 40""","""0575""","""10.02.2022""",null,"""11128240057""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""BERGEN""",206,"""Jorun""","""Barane""",null,2056536,15537,"""M""","""Nedstagata 8""","""5745""",null,null,"""22086139603""",null
"""OSLO""",7,"""John Lewis""","""Baum""",null,2055304,14695,"""Medlem""","""Munkerudveien 81E""","""1165""","""01.01.2019""",null,"""24084528176""","""-- 09.12.2024 Folkeregister op…"
"""BERGEN""",7,"""John Lewis""","""Baum""",null,2055304,14695,"""P""","""Munkerudveien 81E""","""1165""",null,null,"""24084528176""","""-- 09.12.2024 Folkeregister op…"


In [13]:
'''
7. Kompletterer fult uttrekk med følgene oppdateringer for konvertering av dubletter.
	1. Setter inn Kommentarer fra både Oslo og bergen. Oslo først
	2. Setter ogå inn Cornerstone-ID
   Man kan variere på "Menighet" Oslo eller Bergen i søket, Her velger en etter hva slags data en prioriterer. Skal en konvertere medlems dubletter, er det viktig å prioritere hvilken side medlemmene er på Oslo/Bergen.
   Tabellen som skal kommer ut av dette `df_konverterings_tabell_bare_dubletter` benyttes til å Importere i Cornerstone i flere omganger.
	3. Oslo medlemmer
	4. Bergens medlemmer
	5. Dubletter der ingen er medlemmer.
'''

df_konverterings_tabell_bare_dubletter = (
    df_konverterings_tabell
        .join(
            # 1. First join: rename "Cornerstone" to be specific
            df_dublett_komentarer
                .filter(pl.col("Menighet") == "BERGEN") # Endre denne til OSLO eller BERGEN etter som hva man prioriterer av kolonner fra Cornerstone eller Bergens registeret
                .select(
                    [
                        "MedlemID",
                        pl.col("Cornerstone-ID").alias("Cornerstone-ID_Cornerstone"),
                        pl.col("Kommentar").alias("Kommentar_Cornerstone"),
                        pl.col("Fornavn").alias("Fornavn_Cornerstone"),
                        pl.col("Mellomnavn").alias("Mellomnavn_Cornerstone"),
                        pl.col("Etternavn").alias("Etternavn_Cornerstone"),
                        pl.col("Adresse").alias("Adresse_Cornerstone"),
                        pl.col("Postnummer").alias("Postnummer_Cornerstone"),
                        pl.col("Fødselsnummer").alias("Fødselsnummer_Cornerstone"),
                        pl.col("Utmeldt dato 1").alias("Utmeldt dato 1_Cornerstone"),
                        pl.col("Innmeldt dato 1").alias("Innmeldt dato 1_Cornerstone"),
                        
                    ]
                ),
            on="MedlemID",
            #how="left"
        )
    .with_columns(
        pl.concat_str(
            [pl.col("Kommentar_Cornerstone").fill_null(""),
             pl.col("Anmerkning").fill_null("")],
            separator="\n\n"
        ).str.strip_chars().alias("Anmerkning"),
        pl.col("Cornerstone-ID_Cornerstone").alias("Cornerstone-ID"),
        pl.col("Fornavn_Cornerstone").alias("Fornavn"),
        pl.col("Mellomnavn_Cornerstone").alias("Mellomnavn"),
        pl.col("Etternavn_Cornerstone").alias("Etternavn"),
        pl.col("Adresse_Cornerstone").alias("Adresse"),
        pl.col("Postnummer_Cornerstone").alias("Postnummer"),
        pl.col("Fødselsnummer_Cornerstone").alias("Fødselsnummer"),
        pl.col("Utmeldt dato 1_Cornerstone").alias("Utmeldt dato 1"),
        pl.col("Innmeldt dato 1_Cornerstone").alias("Innmeldt dato 1"),
    )
)
# Set the string length globally
#pl.Config.set_fmt_str_lengths(200)

df_konverterings_tabell_bare_dubletter

#pl.Config.set_fmt_str_lengths(50)


MedlemID,Cornerstone-ID,Innmeldingsdato,Er den innmeldte døpt ?,Menighet,Fødested,Dåpsdato,Ant barn under 15 år,Dåpskirke/sted,Fødselsdato,Personnummer,EFTERNAVN,FORNAVN,Stilling,ADRESSE,E-postadresse,POSTNR,Og,SAM_FOR,SAM_EFTER,Utsendelse,Utsendelses kode,Telefon priv,Telefon mob,Sivilstand_Cornerstone,Sivilstand,Trossamfunn,Gift?,Utsendelsesformat,Ektemakens trossamfunn,Anmerkning,Kategori_Cornerstone,Kategori,Giro,RegDato,SamID,FarID,MorID,Kjønn,Svart?,Faddere,Er mor og far gift?,Prest,Dåpsattest,Søk,UtmeldtDato,DødsDato,Kommune,KID-nr,Cornerstone-ID_Cornerstone,Kommentar_Cornerstone,Fornavn_Cornerstone,Mellomnavn_Cornerstone,Etternavn_Cornerstone,Adresse_Cornerstone,Postnummer_Cornerstone,Fødselsnummer_Cornerstone,Utmeldt dato 1_Cornerstone,Innmeldt dato 1_Cornerstone,Fornavn,Mellomnavn,Etternavn,Adresse,Postnummer,Fødselsnummer,Utmeldt dato 1,Innmeldt dato 1
i64,i64,str,bool,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,str,str,str,str,str,str,i64,i64,i64,str,bool,str,bool,str,bool,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
15536,2057856,null,false,null,null,null,null,null,"""07.11.1959""","""07115936284""","""Akerø""","""Janicke""",null,"""Richard Nordraaks gate 9""",null,"""5018""",null,null,null,"""UB""","""B""",null,null,null,null,"""Kristensamfunnet""",false,"""E""",null,"""Felter i fra det gammle Bergen…","""Medlem""","""M""",null,"""20.03.2020""",null,null,13033,null,false,null,false,null,false,null,null,null,null,"""10088100739""",2057856,null,"""Janicke""",null,"""Akerø""","""Richard Nordraaks gate 9""","""5018""","""07115936284""",null,null,"""Janicke""",null,"""Akerø""","""Richard Nordraaks gate 9""","""5018""","""07115936284""",null,null
14800,4681949,"""07.10.2006""",false,"""Bergen""",null,null,null,null,"""22.07.1970""","""22077028874""","""Alsos""","""Gry Veronica""","""Lærer""","""Solåsen 33B""",null,"""5223""",null,null,null,"""UB""","""BS""","""55 10 06 48""",null,null,null,"""Kristensamfunnet""",false,"""E""","""Kristensamfunnet""","""-- 24.08.2023 Folkeregister op…","""Medlem""","""M""",null,"""20.03.2020""",12729,null,null,"""Mann""",false,null,false,null,false,null,null,null,null,"""10171100471""",4681949,"""-- 24.08.2023 Folkeregister op…","""Gry Veronica""",null,"""Alsos""","""Solåsen 33B""","""5223""","""22077028874""",null,"""07.10.2006""","""Gry Veronica""",null,"""Alsos""","""Solåsen 33B""","""5223""","""22077028874""",null,"""07.10.2006"""
12181,3567727,"""22.12.2000""",false,null,null,null,null,null,"""04.08.1947""","""04084728247""","""Aukrust""","""Bodil""",null,"""Haukedalsbotn 39""",null,"""5113""","""og""","""Erik Øystein""","""Hauglin""","""UB""","""B""","""55 18 28 52""",null,null,null,null,false,"""P""",null,"""-- 07.02.2024 folkeregister op…",null,"""V""","""Giro""","""04.02.2022""",null,null,null,"""Kvinne""",false,null,false,null,false,null,null,null,"""BERGEN""",null,3567727,"""-- 07.02.2024 folkeregister op…","""Bodil Elisabeth""",null,"""Aukrust""","""Haukedalsbotn 39""","""5113""","""04084728247""",null,"""22.12.2000""","""Bodil Elisabeth""",null,"""Aukrust""","""Haukedalsbotn 39""","""5113""","""04084728247""",null,"""22.12.2000"""
11780,3567891,null,false,null,null,null,null,null,"""11.12.1982""",null,"""Aukrust""","""Milla Kristine""",null,"""Haukedalsbotn 39""",null,"""5113""",null,null,null,null,null,null,null,null,null,null,false,null,null,"""Felter i fra det gammle Bergen…",null,"""V""","""Giro""","""11.11.2000""",null,null,12181,"""Kvinne""",false,null,false,null,false,null,null,null,"""BERGEN""",null,3567891,null,"""Milla Kristine Hauglin""","""Hauglin""","""Aukrust""","""Haukedalsbotn 39""","""5113""","""11128240057""",null,null,"""Milla Kristine Hauglin""","""Hauglin""","""Aukrust""","""Haukedalsbotn 39""","""5113""","""11128240057""",null,null
15610,3044356,null,false,null,null,null,null,null,null,null,"""Bakken""","""Erling""",null,"""Sanddalsvegen 11 D""",null,